# Notebook Setup


## Install Libraries

In [ ]:
!pip install --quiet langchain_experimental langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.7 MB/s eta 0:00:00


## Set Up environment Variables & API Keys

In [ ]:
from getpass import getpass
OPENAI_API_KEY = getpass("OpenAI API Key: ")

OpenAI API Key: ··········


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

## Import Libraries

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

model_name = 'text-embedding-3-small'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
!pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.0/214.0 kB 3.4 MB/s eta 0:00:00


In [ ]:
from getpass import getpass
PINECONE_API_KEY = getpass("Pinecone API Key: ")
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

Pinecone API Key: ··········


In [ ]:
from pinecone import Pinecone

PINECONE_ENVIRONMENT = "gcp-starter"

index_name = 'work-safe-bc'
pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )
if index_name not in pc.list_indexes().names():
    # we create a new index
    pc.create_index(
        name=index_name,
        metric='cosine',
        dimension=3072
    )

In [ ]:
from langchain.vectorstores import Pinecone

text_field = "text"

# switch back to normal index for langchain
index = pc.Index(index_name)

vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [ ]:
query = "what was the safety guidelines for bakeries?"

vector_search_results = vectorstore.similarity_search_with_score(
    query,  # our search query
    k=3  # return 10 most relevant docs
)
# vector_search_results
print(vector_search_results)

[(Document(page_content='Avoid spreading the flour by hand. Cleaning up\n\t* Do not use compressed air for cleaning. * Do not dry sweep equipment, tabletops, or floors. * Use a HEPA vacuum to remove flour dust from surfaces. * Use a wet mop to clean floors. * Do not let dust accumulate. Clean up spills immediately. * Place waste in covered waste bins. * Take HEPA vacuums outside before changing bags or filters. * Wear a half-face N95 disposable respirator when emptying the HEPA vacuum or changing the filter. Respirators\n* Each worker will be fit-tested if a respirator is required. * The worker must be clean-shaven where the respirator seals with the face. * Used respirators will be discarded at the end of each shift. * When the worker notices a resistance to breathing, the respirator must be replaced. * Respirators will be used in accordance with the respirator program. Safe work planning\n* Work from the Acceptable Control Methods table included in this exposure control plan. * Inspe

In [ ]:
vector_search_results

[(Document(page_content='Avoid spreading the flour by hand. Cleaning up\n\t* Do not use compressed air for cleaning. * Do not dry sweep equipment, tabletops, or floors. * Use a HEPA vacuum to remove flour dust from surfaces. * Use a wet mop to clean floors. * Do not let dust accumulate. Clean up spills immediately. * Place waste in covered waste bins. * Take HEPA vacuums outside before changing bags or filters. * Wear a half-face N95 disposable respirator when emptying the HEPA vacuum or changing the filter. Respirators\n* Each worker will be fit-tested if a respirator is required. * The worker must be clean-shaven where the respirator seals with the face. * Used respirators will be discarded at the end of each shift. * When the worker notices a resistance to breathing, the respirator must be replaced. * Respirators will be used in accordance with the respirator program. Safe work planning\n* Work from the Acceptable Control Methods table included in this exposure control plan. * Inspe


## Initializing the Conversational Agent

Our conversational agent needs a Chat LLM, conversational memory, and a RetrievalQA chain to initialize. We create these using:


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

# chat completion llm
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-4-0125-preview',
    temperature=0.3
)
# conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)
# retrieval qa chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

# retrieval qa chain with sources
from langchain.chains import RetrievalQAWithSourcesChain

qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    return_source_documents=True,
    retriever=vectorstore.as_retriever()
)

### User Queries with LLM responses

In [ ]:
query = "What are the Obligations relating to the use of resource roads?"
qa.invoke(query)

{'query': 'What are the Obligations relating to the use of resource roads?',
 'result': "The obligations relating to the use of resource roads primarily involve ensuring the health and safety of workers and the safe operation of vehicles and equipment on these roads. Here's a summary of the key obligations for both supervisors and workers:\n\n### Supervisor Obligations\nAccording to Section 23 of the Act:\n- Supervisors are required to ensure the health and safety of workers under their supervision.\n- They must make workers aware of foreseeable hazards, which includes providing adequate information on road conditions and hazards.\n- Supervisors are responsible for monitoring worker activity and behavior in using the roads to ensure compliance with safety standards and procedures.\n\n### Worker Obligations\nWorkers using resource roads have several obligations under Section 22 of the Act and specific requirements in the Regulation:\n1. **General Safety**:\n   - Workers must take reason

In [ ]:
query = "What are the Obligations relating to the use of resource roads?"
qa_with_sources.invoke(query)

{'question': 'What are the Obligations relating to the use of resource roads?',
 'answer': 'The obligations relating to the use of resource roads include both supervisor and worker obligations under the Act and specific requirements under the Regulation. Supervisors are required to ensure the health and safety of workers under their supervision, making them aware of foreseeable hazards, including providing adequate information on road conditions and monitoring worker activity and behavior on the roads. Workers must take reasonable care to ensure their own safety and carry out their work in accordance with safe work procedures. This includes following safe operating procedures when operating vehicles, log transporters, and mobile equipment, such as not overtaking another moving industrial vehicle except on a signal from the other vehicle operator, using extreme caution when approaching vehicles from the opposite direction, keeping a safe distance when following crew transportation vehic

## Generation of Evaluation Pairs

In [ ]:
llm_model = 'gpt-4-0125-preview'

In [ ]:
from langchain.evaluation.qa import QAGenerateChain
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI(model=llm_model))

In [ ]:
query = "What are the guidelines relating to the use of resource roads?"

vector_search_results = vectorstore.similarity_search(
    query,  # our search query
    k=10  # return 10 most relevant docs
)
# vector_search_results
print(vector_search_results)

[Document(page_content='2\n. Supervisor obligations\nWorkers using resource roads must also be adequately supervised. Section 23\n of the \nAct\n requires supervisors to ensure the health and safety of workers under their supervision and make those workers aware of foreseeable\nhazards. This obligation would include a number of the activities described above, such as providing adequate information on road conditions and hazards and monitoring worker activity and behaviour in using the roads. 3\n. Worker obligations\nWorkers also have obligations to use roads safely. Primarily, workers must take reasonable care under \nsection 22\n of the \nAct\n to ensure their own safety and carry out their work in accordance with safe work procedures. "Safe\nwork procedures" would include the instruction provided by the employer and/or supervisor for safe use of the road, as well as any traffic control procedures established for the particular road. In addition to the general obligation in the \nAct\

In [ ]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in vector_search_results[:5]]
)

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [ ]:
new_examples[0]["qa_pairs"]

{'query': 'According to the document, what are the obligations of supervisors under Section 23 of the Act regarding the health and safety of workers using resource roads?',
 'answer': 'Supervisors are obligated to ensure the health and safety of workers under their supervision and make those workers aware of foreseeable hazards. This includes providing adequate information on road conditions and hazards, and monitoring worker activity and behavior in using the roads.'}

In [ ]:
examples = [example["qa_pairs"] for example in new_examples]

In [ ]:
qa.invoke(examples[0]["query"])

{'query': 'According to the document, what are the obligations of supervisors under Section 23 of the Act regarding the health and safety of workers using resource roads?',
 'result': 'Under Section 23 of the Act, supervisors are obligated to ensure the health and safety of workers under their supervision and to make those workers aware of foreseeable hazards. This includes activities such as providing adequate information on road conditions and hazards, and monitoring worker activity and behavior in using the resource roads.'}

### Manual Evaluation

In [ ]:
import langchain
langchain.debug = True

In [ ]:
predictions = qa.apply(examples)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `apply` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use batch instead.
  warn_deprecated(


[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "According to the document, what are the obligations of supervisors under Section 23 of the Act regarding the health and safety of workers using resource roads?",
  "answer": "Supervisors are obligated to ensure the health and safety of workers under their supervision and make those workers aware of foreseeable hazards. This includes providing adequate information on road conditions and hazards, and monitoring worker activity and behavior in using the roads."
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "According to the document, what are the obligations of supervisors under Section 23 of the Act regarding the health and safety of workers using resource roads?",
  "context": "2\n. Supervisor obligations\nWorkers us

In [ ]:
from langchain.evaluation.qa import QAEvalChain

In [ ]:
llm = ChatOpenAI(temperature=0, model=llm_model)
eval_chain = QAEvalChain.from_llm(llm)

In [ ]:
graded_outputs = eval_chain.evaluate(examples, predictions)

[chain/start] [1:chain:QAEvalChain] Entering Chain run with input:
{
  "input_list": [
    {
      "query": "According to the document, what are the obligations of supervisors under Section 23 of the Act regarding the health and safety of workers using resource roads?",
      "answer": "Supervisors are obligated to ensure the health and safety of workers under their supervision and make those workers aware of foreseeable hazards. This includes providing adequate information on road conditions and hazards, and monitoring worker activity and behavior in using the roads.",
      "result": "Under Section 23 of the Act, supervisors have the obligation to ensure the health and safety of workers under their supervision and to make those workers aware of foreseeable hazards. This includes activities such as providing adequate information on road conditions and hazards and monitoring worker activity and behavior in using the roads."
    },
    {
      "query": "According to the document, under 

In [ ]:
graded_outputs

[{'results': 'CORRECT'},
 {'results': 'INCORRECT'},
 {'results': 'INCORRECT'},
 {'results': 'CORRECT'},
 {'results': 'INCORRECT'}]

In [ ]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['results'])

Example 0:
Question: According to the document, what are the obligations of supervisors under Section 23 of the Act regarding the health and safety of workers using resource roads?
Real Answer: Supervisors are obligated to ensure the health and safety of workers under their supervision and make those workers aware of foreseeable hazards. This includes providing adequate information on road conditions and hazards, and monitoring worker activity and behavior in using the roads.
Predicted Answer: Under Section 23 of the Act, supervisors have the obligation to ensure the health and safety of workers under their supervision and to make those workers aware of foreseeable hazards. This includes activities such as providing adequate information on road conditions and hazards and monitoring worker activity and behavior in using the roads.
Predicted Grade: CORRECT
Example 1:
Question: According to the document, under what conditions can a prevention officer take action against the engineering ce